# 导入包

In [16]:
import ee 
import geemap
import numpy as np
import pandas as pd
import eerepr
import folium
import os
from functools import partial
geemap.set_proxy(port=10809)
ee.Initialize()

# 解析日期、序号以及shp数据

In [17]:
DataFrame = pd.read_csv(r"E:\Datasets\2020-06-01_2020-09-30_processed\2020-06-01_2020-09-30.csv")
Positive = DataFrame[DataFrame['pos_neg']==1].reset_index(drop=False)
# .dt.date去除分秒
Positive.date = pd.to_datetime(Positive.date).dt.date
Positive['start_date'] = Positive.date - pd.Timedelta(days=20)
Positive['end_date'] =Positive.date + pd.Timedelta(days=20)
# 解析长度
Len_positive = Positive.__len__()

# 按照时间和序号筛选Sentinel-1的SAR数据，在构建的时候同步加载光学影像

In [18]:
# 载入定位用的shp文件
Southest_Tibetan_01 = ee.FeatureCollection('projects/ee-mrwurenzhe/assets/Southest')
# 加入Geo属性信息
Southest_Tibetan_Geo = Southest_Tibetan_01.map(lambda feature:feature.set({'Geo':feature.geometry()}))
# 转为GeometryList
Southest_Tibetan_GeoList = ee.List(Southest_Tibetan_Geo.reduceColumns(ee.Reducer.toList(),['Geo']).get('list'))
Num_list = Southest_Tibetan_GeoList.size().getInfo()

# 需要人工设定的参数,全局变量
CLOUD_FILTER = 60                  # 过滤s2 大于指定云量的数据
Parent_path = r'E:\Datasets' #如不为None，则表示相对路径

In [19]:
# 预定义函数
def make_dir(path):
    isExists = os.path.exists(path)
    # 判断结果
    if not isExists:
        os.makedirs(path)
        print(path + ' 创建成功')
    return path

def clip_AOI(col,AOI):
    return col.clip(AOI)

def time_difference(col,s2_date=None):
    time_difference = ee.Date(s2_date).difference(ee.Date(col.get('system:time_start')), 'days').abs()
    return col.set({'time_difference': time_difference})

def load_image_collection(aoi,start_date,end_date,s2_date):
    # 导入数据，并分为升轨和降轨数据
    s2_sr_col = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)).map(partial(clip_AOI,AOI=aoi)))
    s1_col = (ee.ImageCollection("COPERNICUS/S1_GRD")
        .filterBounds(aoi)
        .filterDate(start_date, end_date).map(partial(clip_AOI,AOI=aoi)))
    s1_col = s1_col.map(partial(time_difference,s2_date=s2_date))
    s1_descending = s1_col.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
    s1_ascending = s1_col.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
    return s1_ascending,s1_descending

def Geemap_export(outdir,image,region,scale=10):
    # 判断升降轨道,s1与s2的数据存储位置相互对应，保存文件仍然是id本身
    Parten = image.get('orbitProperties_pass').getInfo()
    file_name = os.path.join(outdir,Parten+'_'+image.id().getInfo()+'.tif')
    geemap.ee_export_image(image,
                          filename=file_name,
                          scale=scale,region=region,file_per_band=True,timeout=1500)

# 选择最近的一景升轨/降轨SAR影像

In [51]:
for i in range(4337,Len_positive):
    # 获取时间区间以及存储路径
    Obj = Positive.loc[i,['part','num','date','start_date','end_date']]
    start_date = str(Obj.start_date)
    end_date = str(Obj.end_date)
    # 解析路径
    Child_path = os.path.join(str(Positive.loc[i].TimeStart_End),str(Positive.loc[i].part),f'{Positive.loc[i].num:05d}')
    out_dir = os.path.join(Parent_path,Child_path)
    
    # 根据AOI取对应位置和时间范围的S1_Collection
    AOI = ee.Feature.geometry(Southest_Tibetan_GeoList.get(int(Positive.loc[i].num)))
    AOI_buffer = AOI.buffer(distance=300)
    s1_ascending,s1_descending = load_image_collection(AOI_buffer,start_date,end_date,s2_date=str(Obj.date))

    # descending_image && ascending image ，通过将collection转化为list，并定位时间差绝对值最小值序号，获取图像
    ascending_image  = ee.Image(s1_ascending.toList(10).get(int(np.argmin(s1_ascending.aggregate_array('time_difference').getInfo()))))
    descending_image = ee.Image(s1_descending.toList(10).get(int(np.argmin(s1_descending.aggregate_array('time_difference').getInfo()))))
    Geemap_export(out_dir,ascending_image,region=AOI)
    Geemap_export(out_dir,descending_image,region=AOI)
    print('download i={}'.format(i))

Generating URL ...
Please wait ...
Data downloaded to H:\GEE_S2Cloud_Dataset_02\2020-06-01_2020-09-30\Real\03488
Generating URL ...
Please wait ...
Data downloaded to H:\GEE_S2Cloud_Dataset_02\2020-06-01_2020-09-30\Real\03488
download i=839
Generating URL ...
Please wait ...
Data downloaded to H:\GEE_S2Cloud_Dataset_02\2020-06-01_2020-09-30\Real\03490
Generating URL ...
Please wait ...
Data downloaded to H:\GEE_S2Cloud_Dataset_02\2020-06-01_2020-09-30\Real\03490
download i=840
Generating URL ...
Please wait ...
Data downloaded to H:\GEE_S2Cloud_Dataset_02\2020-06-01_2020-09-30\Real\03491
Generating URL ...
Please wait ...
Data downloaded to H:\GEE_S2Cloud_Dataset_02\2020-06-01_2020-09-30\Real\03491
download i=841
Generating URL ...
Please wait ...
Data downloaded to H:\GEE_S2Cloud_Dataset_02\2020-06-01_2020-09-30\Real\03492
Generating URL ...
Please wait ...
Data downloaded to H:\GEE_S2Cloud_Dataset_02\2020-06-01_2020-09-30\Real\03492
download i=842
Generating URL ...
Please wait ...
D

UnboundLocalError: local variable 'r' referenced before assignment

# 追加

In [25]:
index = [2484,6545,1776,7783]
A=[]
for i in index:
    A.append(Positive[Positive.num == i].index[0])
for i in A:
    # 获取时间区间以及存储路径
    Obj = Positive.loc[i,['part','num','date','start_date','end_date']]
    start_date = str(Obj.start_date)
    end_date = str(Obj.end_date)
    # 解析路径
    Child_path = os.path.join(str(Positive.loc[i].TimeStart_End),str(Positive.loc[i].part),f'{Positive.loc[i].num:05d}')
    out_dir = os.path.join(Parent_path,Child_path)
    
    # 根据AOI取对应位置和时间范围的S1_Collection
    AOI = ee.Feature.geometry(Southest_Tibetan_GeoList.get(int(Positive.loc[i].num)))
    AOI_buffer = AOI.buffer(distance=300)
    s1_ascending,s1_descending = load_image_collection(AOI_buffer,start_date,end_date,s2_date=str(Obj.date))

    # descending_image && ascending image ，通过将collection转化为list，并定位时间差绝对值最小值序号，获取图像
    ascending_image  = ee.Image(s1_ascending.toList(10).get(int(np.argmin(s1_ascending.aggregate_array('time_difference').getInfo()))))
    descending_image = ee.Image(s1_descending.toList(10).get(int(np.argmin(s1_descending.aggregate_array('time_difference').getInfo()))))
    Geemap_export(out_dir,ascending_image,region=AOI)
    Geemap_export(out_dir,descending_image,region=AOI)
    print('download i={}'.format(i))

Generating URL ...
Please wait ...
Data downloaded to E:\Datasets\2020-06-01_2020-09-30_processed\Real\02484
Generating URL ...
Please wait ...
Data downloaded to E:\Datasets\2020-06-01_2020-09-30_processed\Real\02484
download i=622
Generating URL ...
Please wait ...
Data downloaded to E:\Datasets\2020-06-01_2020-09-30_processed\Real\06545
Generating URL ...
Please wait ...
Data downloaded to E:\Datasets\2020-06-01_2020-09-30_processed\Real\06545
download i=1756
Generating URL ...
Please wait ...
Data downloaded to E:\Datasets\2020-06-01_2020-09-30_processed\Synth\01776
Generating URL ...
Please wait ...
Data downloaded to E:\Datasets\2020-06-01_2020-09-30_processed\Synth\01776
download i=3049
Generating URL ...
Please wait ...
Data downloaded to E:\Datasets\2020-06-01_2020-09-30_processed\Synth\07783
Generating URL ...
Please wait ...
Data downloaded to E:\Datasets\2020-06-01_2020-09-30_processed\Synth\07783
download i=4305


In [36]:
Positive[(Positive.num == 5792) | (Positive.num ==2770)].index[0]

686

In [10]:
A

[2242, 1541, 686, 1026, 1027, 1025, 3709, 4376, 1110]

In [12]:
Positive[Positive.num == 10931].index[0]

2242

In [39]:
A

[1541, 686, 1026, 1027, 1025, 3708, 4375, 1110]

In [21]:
Positive[Positive.num == 5792 | 2770 | 4354]

,index,Unnamed: 0,parent,TimeStart_End,part,num,tif,pos_neg,date,start_date,end_date


In [ ]:
# 展示以下

In [ ]:
s1_col = (ee.ImageCollection("COPERNICUS/S1_GRD")
    .filterBounds(AOI_buffer)
    .filterDate(start_date,end_date).map(partial(clip_AOI,AOI=AOI_buffer)))

In [ ]:
s1_ascending.aggregate_array('time_difference').getInfo()

In [ ]:
s1_descending.aggregate_array('time_difference').getInfo()